# Project 2 - Source coding, data compression and channel coding

The goal of this second project is to apply some of the principles seen in the lectures about source coding, data compression and channel coding. We ask you to write a brief report (pdf format) collecting your answers to the different questions. All codes must be written in Python inside this Jupyter. Note that you can not change the content of locked cells or import any extra Python library than the ones already imported (numpy).

## Implementation

In this project, you will need to use implement source coding algorithms to answer several questions. Therefore, in this first part, you are asked to write several functions that implement two of the  algorithms seen in the theoretical lectures and one new algorithm described in the project statement. Remember that you need to fill in this Jupyter Notebook to answer these questions. Pay particular attention to the required input and output format of each function.

In [1]:
# [Locked Cell] You can not import any extra Python library in this Notebook.
import numpy as np

### Question 1
Implement a function that returns a binary Huffman code for a given probability distribution. Give the main steps of your implementation. Verify your code on Exercise 7 of the second exercise session (TP2), and report the output of your code for this example. Explain how to extend your function to generate a Huffman code of any (output) alphabet size.


In [9]:
class Node:
    def __init__(self, prob, symbol=None, left=None, right=None):
        self.prob = prob
        self.symbol = symbol
        self.left = left
        self.right = right

def Huffman_code(probability_dict):
    """
    Create the Huffman code for given probabilities

    Arguments:
    ----------
    probability_dict:
      - keys: symbol as character or string
      - values: probability of the symbol as a float or double
      Example: {"A": 0.25, "B":0.5,"C":0.125,"D":0.125}

    Return:
    -------
    - codewords: dictionary with the name and the corresponding codeword
      - keys: symbol as character or string
      - values: associated codeword as a character or a string
      Example: {"A": "10", "B":"0","C":"111","D":"110"}

    """
    
    # Step 1: Create initial list of nodes
    nodes = [Node(prob, symbol) for symbol, prob in probability_dict.items()]

    # Step 2: Manually manage the priority queue
    while len(nodes) > 1:
        # Sort nodes by probability using numpy
        probs = np.array([node.prob for node in nodes])
        sorted_indices = np.argsort(probs)
        nodes = [nodes[i] for i in sorted_indices]

        # Combine two nodes with smallest probabilities
        left = nodes.pop(0)
        right = nodes.pop(0)
        merged = Node(left.prob + right.prob, None, left, right)

        # Add new node back
        nodes.append(merged)

    # Step 3: Traverse the final tree to build the codebook
    def generate_codes(node, prefix="", codebook={}):
        if node.symbol is not None:
            codebook[node.symbol] = prefix
        else:
            generate_codes(node.left, prefix + "0", codebook)
            generate_codes(node.right, prefix + "1", codebook)
        return codebook

    return generate_codes(nodes[0])

probs = {"A": 0.2, "B": 0.2, "C": 0.2, "D": 0.2, "E": 0.2}
codes = Huffman_code(probs)
print(codes)

{'C': '00', 'D': '01', 'E': '10', 'A': '110', 'B': '111'}


### Question 2

Given a sequence of symbols, implement a function that returns a dictionary and the encoded sequence using the on-line Lempel-Ziv algorithm (see State of the art in data compression, slide 50/53). Reproduce and report the example given in the course.

In [ ]:

def LZ_online(sequence):
    """
    The on-line Lempel-Ziv algorithm given a sequence of symbols
    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format

    Return:
    -------
    - dictionary : the computed dictionnary in the form:
        - keys: symbol as character or string
        - values: associated codeword as a tuple composed of the entry index (integer) and a binarized adress with one appended symbol (character or string)
        Example: {'': (0, ''), '0': (1, '0'), '1': (2, '01'), '00': (3, '010'), '10': (4, '100')}
    - encoded_sequence : the encoded sequence in the string format
    """

    dictionary = {"": 0}  # Empty string is always index 0
    encoded_sequence = []
    current_phrase = ""
    index = 1  # Start indexing from 1

    for char in sequence:
        new_phrase = current_phrase + char
        if new_phrase not in dictionary:
            # Add new phrase to dictionary
            encoded_sequence.append((dictionary[current_phrase], char))
            dictionary[new_phrase] = index
            index += 1
            current_phrase = ""
        else:
            current_phrase = new_phrase

    # Handle any remaining phrase
    if current_phrase:
        encoded_sequence.append((dictionary.get(current_phrase[:-1], 0), current_phrase[-1]))

    return dictionary, encoded_sequence

s = "ababcbababaaaaaaa"

dictionary, encoded = LZ_online(s)

print("Encoded sequence:", encoded)

print("Final dictionary:")

for k, v in dictionary.items():
    print(f"{v}: '{k}'")


Encoded sequence: [(0, 'a'), (0, 'b'), (1, 'b'), (0, 'c'), (2, 'a'), (5, 'b'), (1, 'a'), (7, 'a'), (1, 'a')]
Final dictionary:
0: ''
1: 'a'
2: 'b'
3: 'ab'
4: 'c'
5: 'ba'
6: 'bab'
7: 'aa'
8: 'aaa'


### Question 4

Implement a function that returns the encoded sequence using the LZ77 algorithm as described by the algorithm below given an input string and a sliding window size l. Reproduce the example given in Figure 2 with window_size=7.

In [15]:
def LZ77(sequence, window_size=7):
    """
    The Lempel-Ziv 77 algorithm given a sequence of symbols and the sliding window size

    Arguments:
    ----------
    - sequence : Sequence of symbols in the string format
    - window_size : sliding window size as an integer

    Return:
    -------
    - encoded_sequence : the encoded sequence in the string format
    """

    encoded_sequence = []
    i = 0  # Current position in the input string
    
    while i < len(sequence):
        match = (0, 0)  # (offset, length)
        for j in range(max(0, i - window_size), i):
            length = 0
            while (i + length < len(sequence) and
                   sequence[j + length] == sequence[i + length]):
                length += 1
                if j + length >= i:
                    break  # Don't overflow current index
            
            if length > match[1]:
                match = (i - j, length)
        
        # If a match is found
        if match[1] > 0:
            next_char_index = i + match[1]
            next_char = sequence[next_char_index] if next_char_index < len(sequence) else ''
            encoded_sequence.append((match[0], match[1], next_char))
            i += match[1] + 1
        else:
            encoded_sequence.append((0, 0, sequence[i]))
            i += 1

    return encoded_sequence

result = LZ77("abracadabrad", window_size=7)

print("Encoded sequence:")

for triplet in result:
    print(triplet)



Encoded sequence:
(0, 0, 'a')
(0, 0, 'b')
(0, 0, 'r')
(3, 1, 'c')
(5, 1, 'd')
(7, 4, 'd')


In [ ]:
# [Locked Cell] Evaluation of your functions by the examiner.
# You don't have access to the evaluation, this will be done by the examiner.
# Therefore, this cell will return nothing for the students.
import os
if os.path.isfile("private_evaluation.py"):
    from private_evaluation import unit_tests
    unit_tests(Huffman_code, LZ_online, LZ77)

## Source coding and reversible (lossless) data compression


In [6]:
# Write here your codes for questions 5 to 16 (you may delete this comment)

## Channel coding

In [7]:
# Write here your codes for questions 17 to 22 (you may delete this comment)